In [3]:
%%javascript
MathJax.Hub.Config({TeX:{equationNumbers:{autoNumber:"AMS"}}});

<IPython.core.display.Javascript object>

# BCB Final Project - a.y. 17/18

### <i>Fiaschi Lorenzo, Franco Danilo</i>

<ol> <b>TODO</b>
    <li><strike>problem presentation (dataset, objectives)</strike></li>
    <li><strike>preprocessing</strike></li>
    <li><strike>mkl procedure overview</strike></li>
    <li><strike>chosen model description (alignment model, cortes approach)</strike></li>
    <li><strike>algorithm pipeline (with a major point in the cross-validation decisions, how it is shuffled...)</strike></li>
    <li><strike>approach settings</strike></li>
    <li>performances over toy (with description on how it is generated</li>
    <li>integration needed? comparison with single dataset model</li>
    <li>performances over real dataset</li>
    <li>integration needed? comparison with single dataset model</li>
    <li>what we can learn from the best configuration (sparsity)</li>
    <li>kernel integration on the only meaningful dataset (clinical)</li>
    <li>overview on the regression approach</li>
    <li>regression performances with data integration</li>
    <li>single kernel approach after regression results</li>
    <li>final comments over the results</li>
</ol>

### 1. The Problem
The dataset consists of 2741 aligned patients affected by Diabetes; it is structured in 3 differents tables:
<ol>
    <li><b>Genetic Features</b>, 347 - Related to the SNP of 344 genes + 3 composite genes scores [blood pressure, alzheimer, CVD]</li>
    <li><b>Retina's Features</b>, 157 - Engineered features extracted from patients' retinas</li>
    <li><b>Clinical Features</b>, 15 - General patients' information</li>
</ol>
The objective of the study is to find a model that is able to predict whether a patient will incurr in either cardiovascular failures or in episodes of dementia.
In the positive case, it is also aked to predict at which age this episodes will occurr.
In order to learn such model, the output dataset has been provided; it is composed by two information, both for the occurrence of a cardiovascular failure and for the dementia episode: whether the episode has happened or not and at what age.

### 2. Data Preprocessing
In order to be fed, the tables have been cleaned as follows:
<ul>
    <li>
        <b><i>NaN Filling</i></b>: 
        <ul>
            <li>
                Clinical DS: 
                <ul>
                    <li>
                        ["therapy","gender","precedent CVD", "smoker"] $\rightarrow$ most frequent;
                    </li>
                    <li>
                        Others $\rightarrow$ mean.
                    </li>
                </ul>
            <li>
                Genetic DS: 
                <ul>
                    <li>
                        [composite gene scores] $\rightarrow$ mean;
                    </li>
                    <li>
                        Others $\rightarrow$ min.
                    </li>
                </ul>
            <li>
                Vampire DS: 
                <ul>
                    <li>
                        All $\rightarrow$ mean.
                    </li>
                </ul>
        </ul>
    </li>
    <li>
        <b><i>One-Hot Encoding</i></b>:
        <ul>
            <li>Clinical DS:
                <ul>
                    <li>
                        "therapy" and "Apoe4Presence".
                    </li>
                </ul>
            </li>
        </ul>
    </li>
    <li><b><i>Boolean: from binary to symmetric</i></b>: 
        <ul>
            <li>Clinical DS:
                <ul>
                    <li>
                        ["gender","precedent CVD", "smoker"] $\rightarrow$ from [0,1] to [-1,1].
                    </li>
                </ul>
            </li>
            <li>Outputs:
                <ul>
                    <li>
                        ["cvd_fail","dement_fail"] $\rightarrow$ from [0,1] to [-1,1].
                    </li>
                </ul>
            </li>
        </ul>
    </li>
</ul>

### 3. Multiple Kernel Learning
<b>Multiple Kernel Learning</b> refers to a set of machine learning methods that use a predefined set of kernels and learn an optimal linear or non-linear combination of kernels as part of the algorithm.<br>
Reasons to use multiple kernel learning include:
<ul>
    <li>the ability to select for an optimal kernel and parameters from a larger set of kernels, reducing bias due to kernel selection while allowing for more automated machine learning methods;
    </li>
    <li>combining data from different sources (e.g. sound and images from a video) that have different notions of similarity and thus require different kernels;
    </li>
</ul>
<img src='others/mkl.png'>

### 4. The Model
<ul>
    <li><b><i>Target Function</i></b> $\rightarrow$ Similarity-based function; it uses a similarity metric between the combined kernel matrix and an optimum kernel matrix calculated from the training data, in order to select the combination function parameters that maximize the similarity. The similarity between two kernel matrices can be calculated using kernel alignment, Euclidean distance, Kullback-Leibler (KL) divergence, or any other similarity measure.
    </li>
    <li><b><i>Training Method</i></b> $\rightarrow$ One-step method; it calculates both the combination function parameters and the parameters of the combined base learner in a single pass. One can use a sequential approach or a simultaneous approach. In the sequential approach, the combination function parameters are determined first, and then a kernel-based learner is trained using the combined kernel. In the simultaneous approach, both set of parameters are learned together.
    </li>
</ul>
The chosen algorithm is called <i>Centered-kernel alignment</i> [1] and its main purpose is to compute a set of $P$ kernels $K_i$, $i \, = \, 1...P$ from the datasets and to use a linear combination of them to approximate an ideal kernel. Then the obtained approximated kernel can be used for both classification and regression purposes. The functional is:<br>

\begin{equation}
\max\limits_{\eta \in \mathcal{M}} CA(K_{\eta}, IK)
\label{eq:problem}
\end{equation}

where $IK = y^T y$ is the ideal kernel, $K_{\eta} = \sum\limits_{i=1}^P K_i \eta_i$ is its approximation and $\mathcal{M}=\{\eta : ||\eta||_2 = 1\}$ imposing $\eta$ being a unit norm vector. In turn, $CA(K_1, K_2)$ is defined as

\begin{equation*}
CA(K_1^c, K_2^c) = \frac{\langle K_1^c, K_2^c \rangle_F}{\sqrt{\langle K_1^c, K_1^c\rangle_F \; \langle K_2^c, K_2^c \rangle_F}}
\end{equation*}

with $K^c$ is the centered version of $K$ and can be calculated as

\begin{equation*}
K^c = K - \frac{1}{N} 11^TK - \frac{1}{N} K11^T + \frac{1}{N^2} \left(1^TK1\right)11^T
\end{equation*}

where $1$ is the vector of ones with proper dimension.

The optimization problem \eqref{eq:problem} has then a unique analytical solution

\begin{equation}
\eta = \frac{M^{-1}a}{||M^{-1}a||_2}
\label{eq:problem_solution}
\end{equation}

where $M = \{\langle K_m, K_h\rangle_F\}_{m,h \, = \, 1}^P$ and $a = \{\langle K_m, IK\rangle_F\}_{m \, = \, 1}^P$.

In order to avoid overfitting, to be more robust in the learning procedure and in order to get deeper insights of the problem we added either an $L_1$ or $L_2$ penalty to the functional \eqref{eq:problem}, obtaining the following two reformulations

\begin{equation}
\max\limits_{\eta \in \mathcal{M}} CA(K_{\eta}, IK) - \lambda \, ||\eta||_2^2 \;\; (a), \qquad \max\limits_{\eta \in \mathcal{M}} CA(K_{\eta}, IK) - c \, ||\eta||_1 \;\; (b).
\label{eq:problem_penalty}
\end{equation}

While the Equation \eqref{eq:problem_penalty}(a) has the unique analytical solution 

\begin{equation}
\eta = \frac{(M-\lambda I)^{-1}a}{||(M-\lambda I)^{-1}a||_2}
\label{eq:problem_solution_lambda}
\end{equation}

Equation \eqref{eq:problem_penalty}(b) does not due to the non-differentiability of the $L_1$ norm. To converge to the optimum we applied the forward-backward splitting method [2][3].

### 5. The Algorithm

<ol>
    <li><i>Datasets loading</i> $\rightarrow$ (Clinical, Genetic, Vampire, Outputs);</li>
    <li><i>Outer split</i> $\rightarrow$ 75% training + 25% test for final testing purposes;</li>
    <li><i>Kernel definition</i> $\rightarrow$ dictionary list: [{<b>kernelType1:[parameter list]</b>, <b>kernelType2:[parameter list]</b>, <b>...</b> }, <b>...</b> ] </li>
    <li><i>Sampling rounds</i> $\rightarrow$ 3 rounds of 75% training + 25% mid test (extracted from the previous 75% of training) for statistical stability:
        <ol>
            <li>If stated, perform training matrix centering and/or normalisation;</li>
            <li>For each dictionary configuration:
                <ol>
                    <li>Build the ideal kernel from the training outputs;</li>
                    <li>Build all the possible combination between the kernels parameters;</li>
                    <li>3-Fold cross validation over the 75% sampling training:
                        <ol>
                            <li>Build the ideal kernel matrices from both training and validation output sets;</li>
                            <li>For each configurations:
                                <ol>
                                    <li>Compute all the kernel matrices for that configuration;</li>
                                    <li>Find the optimal weights for those kernel matrices using the ideal training kernel;</li>
                                    <li>Compute the weighted sum of the kernel matrices;</li>
                                    <li>Compare the obtained matrix with the ideal validation kernel using the Cortes Alignment metric and store this similarity measure;</li>
                                </ol>
                            </li>
                        </ol>                       
                    </li>
                    <li>Find the configuration with greatest mean across the three validation fold;</li>
                    <li>Recompute weights and similarity score for the newfound setting using the ideal training kernel found at poin <i>a</i>;</li>
                    <li>Compute accuracy, precision and recall for the selected setting against the sampling test set;</li>
                    <li>Store these value, decorated with configuration and alignment for this dictionary;</li>
                </ol>
            </li>
        </ol>
    </li>
    <li>For each sample round, sum all the alignment values related to the same kernel parameters in order to find the configuration that best behaved, keeping the dictionary within a list separated;</li>
    <li>Try the configurations (one per dictionary) against the outer test set, compute accuracy, precision and recall</li>
</ol>

### 6. Approach Setting

Kernel used:
    
<ul>
    <li>Gaussian: $\hspace{4mm}K_{\sigma}(x_i, x) = e^{\,-\,\left(\frac{||x_i \, - \, x||^2}{2\sigma}\right)}$</li>
    <li>Linear: $\hspace{9mm}K(x_i, x) = x_i \cdot x^T$</li>
    <li>Polynomial: $\hspace{1mm}K_d(x_i, x) = (1 +  x_i \cdot x^t)^d$</li>
    <li>Laplacian: $\hspace{3mm}K_{\gamma}(x_i, x) = e^{\,-\,(\, \gamma \, ||x_i \, - \, x||_1)}$</li>
    <li>Sigmoid: $\hspace{5mm}K_{\gamma}(x_i, x) = \tanh \,(\,\gamma \, \langle x_i, x \rangle + 1)$</li>
</ul>

The basical approach exploited has been picking three different kernels and applying them to every dataset, getting back 9 kernel matrices. Several combinations of three data preprocessing methods (Data origin centering, Data Normalization, Kernel Normalization) have been tested.

### 7. Toy Testing

In order to analyze the correctness of the algorithm implemented three synthetic datasets have been generated and the algorithm have been launched over them.


#### 7.1 Toy Dataset Generation

For dataset generation we used the sklearn.datasets.make_classification function [4], initialized with the following configuration:

<ul>
    <li>n_samples = 300</li>
    <li>n_features = 30</li>
    <li>n_informative = 10</li>
    <li>n_redundant = 0</li>
    <li>n_classes = 2</li>
</ul>

The data gathered in this way have been splitted in three datasets and the informative variables have been distributed evenly between the first two. This procedure let us to check if the algorithm is able or not to percieve the irrelevance of the third dataset.

Then, the three datasets have been splitted in training and test sets exploiting sklearn.model_selection.StratifiedShuffleSplit [5]. The initialization in the next

<ul>
    <li>n_splits = 1</li>
    <li>test_size = 0.25</li>
</ul>


#### 7.2 Performances over Toy

##### 7.1.1 MKL

In the next the results achieved using the Toy datasets are shown. In particular, they are compared with the learning performances got studying one single dataset (the first one) using common learning algorithm, e.g., Logistic Regression and SVM.

<table style="width:100%">
    <tr>
        <th> 
          <p><font  color="red">Based on CA</font></p>
        </th>
        <th align="justify">
          Configuration
        </th>
        <th>
          Test
        </th>
        <th>
          Train
        </th>
        <th>
          Eta
        </th>
    </tr>
    <tr>
        <th>
            L2 - Centering -$\;\;$ Normalizing
        </th>
        <td align="left">
            Lambda: 1.3<br>
            Laplacian : 0.2, 0.6, 0.2<br>
            Gaussian: 0.3, 0.3, 0.6
        </td>
        <td>
            Accuracy: 0.8549075391180654<br>
            Precision: 0.782608695652174<br>
            Recall: 0.972972972972973
        </td>
        <td>
            CA: 0.21867020939197357<br>
            Accuracy: 0.8483169129720854<br>
            Precision: 0.8315456327135972<br>
            Recall: 0.8690476190476191
        </td>
        <td>
            [0.11894472, -0.10650123, -0.58678192, 0.38271258,  0.64068699, -0.27064346]
        </td>
    </tr>
    <tr>
        <th>
            L2 - Centering -$\;$K Normalizing
        </th>
        <td>
            Lambda: 0.7<br>
            Polynomial : 2, 2, 7<br>
            Gaussian: 0.3, 0.3, 0.6
        </td>
        <td>
            Accuracy: 0.8275248933143671<br>
            Precision: 0.7857142857142857<br>
            Recall: 0.8918918918918919
        </td>
        <td>
            CA: 0.2029201672369875<br>
            Accuracy: 0.8185550082101806<br>
            Precision: 0.8212698412698413<br>
            Recall: 0.8095238095238094
        </td>
        <td>
            [0.69048272, -0.40438296, 0.33601634, 0.23292882, 0.41944434, 0.12888623]
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering - $\;\;$ Normalizing
        </th>
        <td>
            Lambda: 0.5<br>
            Linear : -<br>
            Gaussian: 0.3, 0.3, 0.3
        </td>
        <td>
            Accuracy: 0.8410384068278806<br>
            Precision: 0.7906976744186046<br>
            Recall: 0.918918918918919
        </td>
        <td>
            CA: 0.2448483167916586<br>
            Accuracy: 0.8130131362889984<br>
            Precision: 0.8050719690504637<br>
            Recall: 0.8214285714285715
        </td>
        <td>
            [0.15587906, 0.00148777, 0.27568084, 0.00145643, 0.94852263, 0.00151153]
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering -$\;$K Normalizing
        </th>
        <td>
            Lambda: 0.5<br>
            Polynomial: 3, 3, 3<br>
            Gaussian: 0.5, 0.5, 0.1
        </td>
        <td>
            Accuracy: 0.893669985775249<br>
            Precision: 0.8717948717948718<br>
            Recall: 0.918918918918919
        </td>
        <td>
            CA: 0.24299832814264288<br>
            Accuracy: 0.7545155993431857<br>
            Precision: 0.7457902298850575<br>
            Recall: 0.7619047619047619
        </td>
        <td>
            [0.25203873, 0.44604029, 0.25542251, 0.44868187, 0.51932414, 0.44863225]
        </td>
    </tr>
</table>

<br>
<br>

<table style="width:100%">
    <tr>
        <th>
          <p><font  color="red">Based on Accuracy</font></p>
        </th>
        <th align="justify">
          Configuration
        </th>
        <th>
          Test
        </th>
        <th>
          Train
        </th>
        <th>
          Eta
        </th>
    </tr>
    <tr>
        <th>
            L2 - Centering - $\;\;$ Normalizing
        </th>
        <td align="left">
            Lambda: 1.3<br>
            Linear: -<br>
            Polynomial : 2, 2, 2<br>
            Gaussian: 0.1, 0.1, 0.5
        </td>
        <td>
            Accuracy: 0.8669985775248933<br>
            Precision: 0.8461538461538461<br>
            Recall: 0.8648648648648649
        </td>
        <td>
            CA: 0.22131619925232068<br>
            Accuracy: 0.7779146141215106<br>
            Precision: 0.7674603174603174<br>
            Recall: 0.7857142857142856
        </td>
        <td>
            [0.04692887, -0.06199509, -0.53220357, 0.01056036, -0.02753925, 0.82368748, 0.018547, 0.07773182,  0.15810262]
        </td>
    </tr>
    <tr>
        <th>
            L2 - Centering -$\;$K Normalizing
        </th>
        <td>
            Lambda: 0.7<br>
            Linear: -<br>
            Polynomial : 2, 2, 2<br>
            Gaussian: 0.5, 0.1, 0.7
        </td>
        <td>
            Accuracy: 0.8538406827880511<br>
            Precision: 0.825<br>
            Recall: 0.8918918918918919
        </td>
        <td>
            CA: 0.21750315770171394<br>
            Accuracy: 0.7955665024630543<br>
            Precision: 0.781007115489874<br>
            Recall: 0.8095238095238094
        </td>
        <td>
            [0.33429467, -0.17592443, -0.39955222, 0.15408311, -0.10726992, 0.10759013, 0.07914564,  0.71947001,  0.35627305]
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering - $\;\;$ Normalizing
        </th>
        <td>
            Lambda: 0.9<br>
            Linear: -<br>
            Gaussian: 0.3, 0.6, 0.3
        </td>
        <td>
            Accuracy: 0.8534850640113798<br>
            Precision: 0.8421052631578947<br>
            Recall: 0.8648648648648649
        </td>
        <td>
            CA: 0.20859956307049451<br>
            Accuracy: 0.8013136288998357<br>
            Precision: 0.7910509031198686<br>
            Recall: 0.8095238095238094
        </td>
        <td>
            [0.18207216, 0.00148136, 0.26428069, 0.00133108, 0.94710043, 0.00150188]
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering -$\;$K Normalizing
        </th>
        <td>
            Lambda: 0.5<br>
            Sigmoid: 0.4, 0.9, 0.4<br>
            Gaussian: 0.5, 0.5, 1
        </td>
        <td>
            Accuracy: 0.8403271692745378<br>
            Precision: 0.8205128205128205<br>
            Recall: 0.8648648648648649
        </td>
        <td>
            CA: 0.21444242841302316<br>
            Accuracy: 0.8423645320197045<br>
            Precision: 0.8346319628647215<br>
            Recall: 0.8571428571428571
        </td>
        <td>
            [0.28083217, 0.457611, 0.25571453, 0.46127277, 0.47375624, 0.45729422]
        </td>
    </tr>
</table>


##### 7.1.2 L1 Logistic Regression
<br>
<table style="width:100%">
    <tr>
        <th>
          <p><font  color="red">Based on Logistic Regression</font ></p>
        </th>
        <th align="justify">
          Best Lambda
        </th>
        <th>
          Test
        </th>
        <th>
          Eta
        </th>
    </tr>
    <tr>
        <th>
            <p>L1 - Centering - $\;\;$ Normalizing</p>
        </th>
        <td align="left">
            0.1
        </td>
        <td>
            Accuracy: 0.5472972972972974<br>
            Precision: 0.5365853658536586<br>
            Recall: 0.5945945945945946
        </td>
        <td>
            [0, 0, 0, 0, 0, 0, 0.70943101, 0, 0, 0]
        </td>
    </tr>
    <tr>
        <th>
            <p>L1 - Centering</p>
        </th>
        <td align="left">
            1.9
        </td>
        <td>
            Accuracy: 0.5472972972972974<br>
            Precision: 0.5365853658536586<br>
            Recall: 0.5945945945945946
        </td>
        <td>
            [0, 0, 0, 0, 0, 0, 0.70943101, 0, 0, 0]
        </td>
    </tr>
</table>

#### 7.3 Results discussion

Some considerations can be made out of the results:
<ul>
    <li>it seems to be some performances improvements using an approach based on the CA rather than one based on the accuracy;</li>
    <li>the results obtained using kernel normalisation approaches are generally comparable, in term of accuracy, with the one achieved by the normalisation procedure usage;</li>
    <li>the selected kernels are generally belonging to the set K = {Linear, Polynomial};</li>
    <li>even if the informative features are not evenly distributed among the datasets (in particulare none of them is in the third one), looking at the $\eta$ coefficients, the algorithm seems not to see it. Indeed the coefficients magnitudes, kernel-wise, are generally comparable (also in the best performances);</li>
    <li>compared with the shallow learning realized on only the first dataset, MKL's performances are notably higher. This is in line with the fact that such dataset contains only the 70% of the global, useful information.</li>
</ul>

### 8 Real Data

In the next, the results achieved over the real data are shown and discussed. Again the comparison between the data integration approach and the shallow learning is proposed.


#### 8.1 Performances 

##### 8.1.1 MKL

<table style="width:100%">
    <tr>
        <th>
          <p><font  color="red">Based on CA</font ></p>
        </th>
        <th align="justify">
          Cardio configuration
        </th>
        <th>
          Cardio test
        </th>
        <th>
          Cardio train
        </th>
        <th>
          Eta
        </th>
    </tr>
    <tr>
        <th>
            L2 - Centering - $\;\;$ Normalizing
        </th>
        <td align="left">
            Lambda: 1.3<br>
            Linear: - <br>
            Gaussian: 0.3, 0.6, 0.6
        </td>
        <td>
            Accuracy: 0.5142017160805945<br>
            Precision: 0.34534534534534533<br>
            Recall: 0.5043859649122807
        </td>
        <td>
            CA: 0.022069836608639065<br>
            Accuracy: 0.5173198410656604<br>
            Precision: 0.3762802870865037<br>
            Recall: 0.18343195266272191
        </td>
        <td>
            [1.53594173e-01, 2.05124331e-04, 9.88133950e-01, 2.05128456e-04, 6.19603791e-08, 2.05128150e-04]
        </td>
    </tr>
    <tr>
        <th>
            L2 - Centering -$\;$K Normalizing
        </th>
        <td align="left">
            Lambda: 0.7<br>
            Polynomial: 5, 3, 5<br>
            Gaussian: 1, 1, 1
        </td>
        <td>
            Accuracy: 0.6335995556577032<br>
            Precision: 0.44857142857142857<br>
            Recall: 0.6885964912280702
        </td>
        <td>
            CA: 0.021912127304593684<br>
            Accuracy: 0.5151302061001058<br>
            Precision: 0.3495905848609797<br>
            Recall: 0.4437869822485207
        </td>
        <td>
            [9.99995428e-01, 1.55040582e-03, 4.85827949e-04, 1.73244483e-03, -7.08555105e-04, 1.73244188e-03]
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering - $\;\;$ Normalizing
        </th>
        <td align="left">
            Lambda: 0.9<br>
            Polynomial: 7, 7, 2<br>
            Gaussian: 0.3, 0.3, 0.3
        </td>
        <td>
            Accuracy: 0.5172182640006129<br>
            Precision: 0.3534136546184739<br>
            Recall: 0.38596491228070173
        </td>
        <td>
            CA: 0.018437225730988282<br>
            Accuracy: 0.5750821827744905<br>
            Precision: 0.39044558794638135<br>
            Recall: 0.6390532544378699
        </td>
        <td>
            [8.61000942e-09, 5.92773226e-25, 2.09550805e-23, 5.92772271e-25, 1.00000000e+00, 5.92772251e-25]
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering -$\;$K Normalizing
        </th>
        <td align="left">
            Lambda: 0.9<br>
            Polynomial: 5, 3, 3<br>
            Gaussian: 0.5, 1, 0.5
        </td>
        <td>
            Accuracy: 0.552325135984065<br>
            Precision: 0.39114391143911437<br>
            Recall: 0.4649122807017544
        </td>
        <td>
            CA: 0.02140327827986396<br>
            Accuracy: 0.5059085841694538<br>
            Precision: 0.3522058823529412<br>
            Recall: 0.15384615384615385
        </td>
        <td>
            [0.00423997, 0.57731552, 0.00998617, 0.57731614, -0.00111829, 0.57731613]
        </td>
    </tr>
</table>

<table style="width:100%">
    <tr>
        <th>
          <p><font  color="red">Based on CA</font></p>
        </th>
        <th>
          Dementia configuration
        </th>
        <th>
          Dementia test
        </th>
        <th>
          Dementia train
        </th>
        <th>
          Eta
        </th>
    </tr>
    <tr>
        <th>
            L2 - Centering - $\;\;$ Normalizing
        </th>
        <td align="left">
            Lambda: 0.9<br>
            Polynomial: 2, 7, 7<br>
            Gaussian: 0.3, 0.3, 0.3
        </td>
        <td>
            Accuracy: 0.5015561780267662<br>
            Precision: 0.25<br>
            Recall: 0.008403361344537815
        </td>
        <td>
            CA: 0.019553242196465082<br>
            Accuracy: 0.5292355144304913<br>
            Precision: 0.22729634621144057<br>
            Recall: 0.3408239700374532
        </td>
        <td>
            [5.27925090e-51, 0.00000000e+00, 1.31633288e-53, 4.87947698e-56, 1.00000000e+00, 4.87947064e-56]
        </td>
    </tr>
    <tr>
        <th>
            L2 - Centering -$\;$K Normalizing
        </th>
        <td align="left">
            Lambda: 0.7<br>
            Polynomial: 3, 3, 3<br>
            Gaussian: 0.5, 0.5, 0.5
        </td>
        <td>
            Accuracy: 0.6961821765743335<br>
            Precision: 0.2824858757062147<br>
            Recall: 0.8403361344537815
        </td>
        <td>
            CA: 0.02090113112000945<br>
            Accuracy: 0.47510905485789817<br>
            Precision: 0.15422840892082024<br>
            Recall: 0.37453183520599254
        </td>
        <td>
            [9.99980246e-01, -2.49677856e-03, 5.97825819e-04, -4.00966828e-03, -8.51582861e-04, -4.01427793e-03]
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering - $\;\;$ Normalizing
        </th>
        <td align="left">
            Lambda: 0.7<br>
            Linear: -<br>
            Gaussian: 0.3, 0.6, 0.3
        </td>
        <td>
            Accuracy: 0.5016599232285507<br>
            Precision: 0.18<br>
            Recall: 0.07563025210084033
        </td>
        <td>
            CA: 0.019806779612404655<br>
            Accuracy: 0.6351398986560916<br>
            Precision: 0.24566315181426832<br>
            Recall: 0.7565543071161048
        </td>
        <td>
            [3.84334703e-09, 1.08289854e-11, 1.32260026e-09, 1.08289422e-11, 1.00000000e+00, 1.08289425e-11]
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering -$\;$K Normalizing
        </th>
        <td align="left">
            Lambda: 0.5<br>
            Polynomial: 2, 2, 2<br>
            Gaussian: 0.6, 0.6, 0.3
        </td>
        <td>
            Accuracy: 0.5398900300861085<br>
            Precision: 0.19648093841642228<br>
            Recall: 0.5630252100840336
        </td>
        <td>
            CA: 0.020095487569578376<br>
            Accuracy: 0.51694646397885<br>
            Precision: 0.18319912253818735<br>
            Recall: 0.5131086142322098
        </td>
        <td>
            [0.00191554, 0.57733907, 0.00581971, 0.57733679, 0.00267856, 0.57733623]
        </td>
    </tr>
</table>

<table style="width:100%">
    <tr>
        <th>
          <p><font  color="red">Based on Accuracy</font></p>
        </th>
        <th align="justify">
          Cardio configuration
        </th>
        <th>
          Cardio test
        </th>
        <th>
          Cardio train
        </th>
        <th>
          Eta
        </th>
    </tr>
    <tr>
        <th>
            L2 - Centering - $\;\;$ Normalizing
        </th>
        <td align="left">
            Lambda: 1.3<br>
            Polynomial: 7, 2, 2<br>
            Gaussian: 0.3, 0.6, 0.3
        </td>
        <td>
            Accuracy: 0.5105611601513241<br>
            Precision: 0.3680297397769517<br>
            Recall: 0.4057377049180328
        </td>
        <td>
            CA: 0.016353709305238603<br>
            Accuracy: 0.561161760875228<br>
            Precision: 0.3639260853392847<br>
            Recall: 0.703030303030303
        </td>
        <td>
            [1.00000000e+00, 1.11659997e-27, 3.61028219e-27, 1.11655594e-27, 1.92671234e-14, 1.11654064e-27]
        </td>
    </tr>
    <tr>
        <th>
            L2 - Centering -$\;$K Normalizing
        </th>
        <td>
            Lambda: 0.9<br>
            Polynomial: 5, 3, 3<br>
            Gaussian: 0.5, 0.5, 1
        </td>
        <td>
            Accuracy: 0.6089032712706772<br>
            Precision: 0.45027624309392267<br>
            Recall: 0.6680327868852459
        </td>
        <td>
            CA: 0.020394238107195952<br>
            Accuracy: 0.535972909611878<br>
            Precision: 0.3564915954553502<br>
            Recall: 0.496969696969697
        </td>
        <td>
            [9.99997666e-01, -9.75757165e-04, -1.48960098e-03, -8.54729364e-04, 4.36034115e-05, -8.73927921e-04]
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering - $\;\;$ Normalizing
        </th>
        <td>
            Lambda: 1.3<br>
            Laplacian: 0.2, 0.2, 0.6<br>
            Gaussian: 0.3, 0.3, 0.3
        </td>
        <td>
            Accuracy: 0.5385449892441214<br>
            Precision: 0.39928057553956836<br>
            Recall: 0.45491803278688525
        </td>
        <td>
            CA: 0.019821356086981313<br>
            Accuracy: 0.5902955051952187<br>
            Precision: 0.3955692150083238<br>
            Recall: 0.6505050505050506
        </td>
        <td>
            [0.41148454, 0.40759869, 0.40763828, 0.40758741, 0.40758505, 0.40758035]
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering -$\;$K Normalizing
        </th>
        <td>
            Lambda: 1.3<br>
            Laplacian: 0.6, 0.2, 0.2<br>
            Gaussian: 0.6, 0.3, 0.3
        </td>
        <td>
            Accuracy: 0.5208534233365477<br>
            Precision: 0.37537537537537535<br>
            Recall: 0.5122950819672131
        </td>
        <td>
            CA: 0.025973758304765438<br>
            Accuracy: 0.5408092385169749<br>
            Precision: 0.3452091152501549<br>
            Recall: 0.7272727272727272
        </td>
        <td>
            [0.40824884, 0.40823975, 0.40826687, 0.40824452, 0.40825236, 0.4082374]
        </td>
    </tr>
</table>

<table style="width:100%">
    <tr>
        <th>
          <p><font  color="red">Based on Accuracy</font></p>
        </th>
        <th align="justify">
          Dementia configuration
        </th>
        <th>
          Dementia test
        </th>
        <th>
          Dementia train
        </th>
        <th>
          Eta
        </th>
    </tr>
    <tr>
        <th>
            L2 - Centering - $\;\;$ Normalizing
        </th>
        <td align="left">
            Lambda: 0.5<br>
            Sigmoid: 0.6, 0.2, 0.6 <br>
            Gaussian: 0.3, 0.6, 0.6
        </td>
        <td>
            Accuracy: 0.5011411972196286<br>
            Precision: 0.173841059602649<br>
            Recall: 0.8823529411764706
        </td>
        <td>
            CA: 0.022936515148907655<br>
            Accuracy: 0.5318572372769332<br>
            Precision: 0.18846890611201905<br>
            Recall: 0.8127340823970037
        </td>
        <td>
            [0.00000000e+00, -5.98992292e-01, 3.88679462e-07, 5.66845714e-01, 0.00000000e+00, 5.65591876e-01]
        </td>
    </tr>
    <tr>
        <th>
            L2 - Centering -$\;$K Normalizing
        </th>
        <td>
            Lambda: 0.5<br>
            Polynomial: 5, 3, 3 <br>
            Gaussian: 1, 0.5, 1
        </td>
        <td>
            Accuracy: 0.637410519763461<br>
            Precision: 0.24793388429752067<br>
            Recall: 0.7563025210084033
        </td>
        <td>
            CA: 0.023585630883914677<br>
            Accuracy: 0.5367614011896894<br>
            Precision: 0.19454027390545056<br>
            Recall: 0.5543071161048689
        </td>
        <td>
            [9.99997434e-01, 1.09082797e-03, -1.14841501e-03, 1.10519947e-03, 4.54430047e-04, 1.09300865e-03]
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering - $\;\;$ Normalizing
        </th>
        <td>
            Lambda: 0.5<br>
            Sigmoid: 0.6, 0.2, 0.2 <br>
            Gaussian: 0.3, 0.3, 0.3
        </td>
        <td>
            Accuracy: 0.5039423176678078<br>
            Precision: 0.18292682926829268<br>
            Recall: 0.12605042016806722
        </td>
        <td>
            CA: 0.02304056615523532<br>
            Accuracy: 0.6506895792024675<br>
            Precision: 0.2543905061013978<br>
            Recall: 0.7790262172284644
        </td>
        <td>
            [0, 0.57739941, 0, 0.57732492, 0, 0.57732647]
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering -$\;$K Normalizing
        </th>
        <td>
            Lambda: 0.5<br>
            Sigmoid: 0.9, 0.4, 0.4 <br>
            Gaussian: 0.5, 0.5, 1
        </td>
        <td>
            Accuracy: 0.5420686793235813<br>
            Precision: 0.19877675840978593<br>
            Recall: 0.5462184873949579
        </td>
        <td>
            CA: 0.0235832075723479<br>
            Accuracy: 0.5681824190350298<br>
            Precision: 0.21179408302001446<br>
            Recall: 0.5355805243445693
        </td>
        <td>
            [0, 0.57735433, 0, 0.57734834, 0, 0.57734814]
        </td>
    </tr>
</table>
<br><br>

##### 8.1.2 L1 Logistic Regression
<br>
<table style="width:100%">
    <tr>
        <th>
          <p><font  color="red">Based on Logistic Regression</font ></p>
        </th>
        <th align="justify">
          Cardio Best Lambda
        </th>
        <th>
          Cardio test
        </th>
        <th>
          Eta
        </th>
    </tr>
    <tr>
        <th>
            <p>L1 - Centering - $\;\;$ Normalizing</p>
        </th>
        <td align="left">
            1.9
        </td>
        <td>
            Accuracy: 0.5846452922699763<br>
            Precision: 0.5754716981132075<br>
            Recall: 0.2675438596491228
        </td>
        <td>
            [0.24295029, 0, 2.20323597, -0.75412797, 0.23335976, 0.52585566, 0, 0, 0.71051908, 2.84089014, -1.29482284, 2.3777739, -3.18657635, 8.22248962, 0, -0.10014134, 0, 0.51144475, 0.82023147, 0, 1.18526461]
        </td>
    </tr>
    <tr>
        <th>
            <p>L1 - Centering</p>
        </th>
        <td align="left">
            1.9
        </td>
        <td>
            Accuracy: 0.6411265609438443<br>
            Precision: 0.4723127035830619<br>
            Recall: 0.6359649122807017
        </td>
        <td>
            [0.24541592, 0, 2.20349362, -0.75647233,  0.23339294,  0.52580679, 0, 0, 0.71058628,  2.84110754, -1.29468123,  2.3778646, -3.18673149, 8.22266713, 0, -0.09714606, 0, 0.50942805, 0.81592509, 0, 1.18512638]
        </td>
    </tr>
</table>
<br>
<br>
<table style="width:100%">
    <tr>
        <th>
          <p><font  color="red">Based on Logistic Regression</font ></p>
        </th>
        <th align="justify">
          Dementia Best Lambda
        </th>
        <th>
          Dementia test
        </th>
        <th>
          Eta
        </th>
    </tr>
    <tr>
        <th>
            <p>L1 - Centering - $\;\;$ Normalizing</p>
        </th>
        <td align="left">
            0.5
        </td>
        <td>
            Accuracy: 0.5048760244838676<br>
            Precision: 0.3333333333333333<br>
            Recall: 0.01680672268907563
        </td>
        <td>
            [0, 0, 0, -0.34944809, 0.47924946, -4.33491868, 0, 0, 1.68668473, 0, 0, 0, 0, -0.1756032, 0, 0, 0, 0, 0, 0]
        </td>
    </tr>
    <tr>
        <th>
            <p>L1 - Centering</p>
        </th>
        <td align="left">
            0.5
        </td>
        <td>
            Accuracy: 0.6715945637514265<br>
            Precision: 0.27575757575757576<br>
            Recall: 0.7647058823529411
        </td>
        <td>
            [0, 0, 0, -0.34944809, 0.47924946, -4.33491868, 0, 0, 1.68668473, 0, 0, 0, 0, 0, -0.1756032, 0, 0, 0, 0, 0, 0,]
        </td>
    </tr>
</table>
<br>

#### 8.2 Results discussion
Some considerations can be made out of the results:
<ul>
    <li>it seems to be some performances improvements using an approach based on the CA rather than one based on the accuracy</li>
    <li>the kernel normalisation approach gives better results in term of accuracy when compared to the others;</li>
    <li>usually the approach based on the L2 penalization picks a Polynomial kernel;</li>
    <li>the kernel normalisation provides always the best result when the functional is penalized with an L2-norm Tikhonov;</li>
    <li>the selected kernel tends to belong to set K = {Laplacian, Sigmoid, Polynomial};</li>
    <li>among the best performances, the Polynomial kernel is always selected and the clinical dataset is pointed out as the most informative; the latter is justified by the coefficient ($\eta_1$) associated to such dataset once transformed by the Polynomial kernel. Indeed it is several orders of magnitude greater than the others coefficients;</li>
    <li>due to the fact that the information is mainly contained in the clinical dataset, the performances achieved with a shallow classification are comparable with the ones got exploiting MKL.</li>
</ul>

### 9 Further considerations and studies

From the Section 8 has been inferred that with high probability the most (and only!) meaningful dataset for the project purposes (see Section 1) is the clinical one. Because of this an interesting study could have been the one which applies a multi-kernel learning procedure only to such dataset. This kind of approach let the realization of multiple points of view (one per used kernel) of the same object, i.e., the clinical dataset. Such procedure could provide a better comprehension of the correlation existing between the data and the diseases.


#### 9.1 MKL over the Clinical dataset

Looking at the results of Section 8, has been decided to generate the wider overall vision of the Clinical dataset exploiting the Polynomial kernel both along with the Gaussian one (again, but with a larger set of possible meta-parameters) and the other ones. In the next the performances.

<table style="width:100%">
    <tr>
        <th>
          <p><font  color="red">Based on Lasso and SVM</font ></p>
        </th>
        <th align="justify">
          Cardio Best Lambda
        </th>
        <th>
          Cardio test
        </th>
        <th>
          Eta
        </th>
    </tr>
    <tr>
        <th>
            <p>L2 - Centering - $\;\;$ Normalizing</p>
        </th>
        <td align="left">
            1.9
        </td>
        <td>
            Average Error: 0.5846452922699763<br>
            Error Variance: 0.575471698113207
        </td>
        <td>
            -
        </td>
    </tr>
    <tr>
        <th>
            <p>L2 - Centering</p>
        </th>
        <td align="left">
            1.9
        </td>
        <td>
            Accuracy: 0.6411265609438443<br>
            Precision: 0.4723127035830619<br>
            Recall: 0.6359649122807017
        </td>
        <td>
            -
        </td>
    </tr>
    <tr>
        <th>
            <p>L1 - Centering - $\;\;$ Normalizing</p>
        </th>
        <td align="left">
            0.01
        </td>
        <td>
            Average Error: 0.8382311045954134<br>
            Error Variance: 0.11874280646397932
        </td>
        <td>
            [0, 0,  0, 0.14386508, 0.05589937, 0, 0, 0, 0.25778769  0, 0,  0, 0, 0.99383314, 0, 0, 0,  0, 0, 0, 0]
        </td>
    </tr>
    <tr>
        <th>
            <p>L1 - Centering</p>
        </th>
        <td align="left">
            0.01
        </td>
        <td>
            Average Error: 2.779940546323388<br>
            Error Variance: 4.34040204773008
        </td>
        <td>
            [0, 0,  0, 0.14386508, 0.05589937, 0, 0, 0, 0.25778769  0, 0,  0, 0, 0.99383314, 0, 0, 0,  0, 0, 0, 0]
        </td>
    </tr>
</table>
<br>
<br>
<table style="width:100%">
    <tr>
        <th>
          <p><font  color="red">Based on Lasso and SVM</font ></p>
        </th>
        <th align="justify">
          Dementia Best Lambda
        </th>
        <th>
          Dementia test
        </th>
        <th>
          Eta
        </th>
    </tr>
    <tr>
        <th>
            <p>L2 - Centering - $\;\;$ Normalizing</p>
        </th>
        <td align="left">
            0.12
        </td>
        <td>
            Average error: 0.6938040471139271<br>
            Error variance: 0.711149051596936
        </td>
        <td>
            -
        </td>
    </tr>
    <tr>
        <th>
            <p>L2 - Centering</p>
        </th>
        <td align="left">
            0.12
        </td>
        <td>
            Average error: 0.6938040471139271<br>
            Error variance: 0.711149051596936
        </td>
        <td>
            -
        </td>
    </tr>
    <tr>
        <th>
            <p>L1 - Centering - $\;\;$ Normalizing</p>
        </th>
        <td align="left">
            0.5
        </td>
        <td>
            Average error: 0.8432866613431059<br>
            Error variance: 0.13348983190138347
        </td>
        <td>
            [0, 0, 0, -0.21727138  0.07391699, 0, 0, 0,  0.26572281, 0, 0, 0, 0, 1.20944333, 0, 0, 0, 0, 0, 0, 0]
        </td>
    </tr>
    <tr>
        <th>
            <p>L1 - Centering</p>
        </th>
        <td align="left">
            0.5
        </td>
        <td>
            Average error: 3.3485817029263245<br>
            Error variance: 6.033243528951659
        </td>
        <td>
            [0, 0, 0, -0.21727138  0.07391699, 0, 0, 0,  0.26572281, 0, 0, 0, 0, 1.20944333, 0, 0, 0, 0, 0, 0, 0]
        </td>
    </tr>
</table>


#### 9.2 Results discussion

### 10 Shallow regression

Once selected as the most (and only) relevant dataset for dementia and cardiovascular failures, the Clinical one has been exploited for regression purposes. The regressive learning has the goal to predict the age at which the diesease will arise.
The regression algorithm adopted are Lasso and SVM. In the next the performances and their discussion.


#### 10.1 Shallow regression performances



#### 10.2 Results discussion

### 11 Conclusions and further possible studies

In this section is presented a summarization of the results that have been obtained during the study presented above. Further possible study are suggested. Most of the ideas have not been tested largely due to computational limits of the authors.


#### 11.1 What has been learned

In the context of the Cortes Alignment, the Polynomial kernel usage seems to constantly provides higher performances<br>

#### 11.2 Best configurations


#### 11.3 Suggested further studies

Preprocessing with filter methods<br>
Configurations performance stability analysis<br>
Configurations performance over different approaches analysis<br>
Changing of the functional using a risk minimizator reather than a kernel aligner<br>

### References
[1] Corinna Cortes, Mehryar Mohri, and Rostamizadeh Afshin (2010). "Two-stage learning kernel algorithms", In Proceedings of the 27th International Conference on Machine Learning.<br>
[2] https://en.wikipedia.org/wiki/Proximal_gradient_methods_for_learning<br>
[3] Combettes, Patrick L.; Wajs, Valérie R. (2005). "Signal Recovering by Proximal Forward-Backward Splitting", Multiscale Model, Simul 4 (4): 1168–1200. doi:10.1137/050626090<br>
[4] http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html<br>
[5] http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html